In [328]:
import pandas as pd
import os
import re

In [329]:
base_dir = os.getcwd() 

ruta_grados = os.path.join(base_dir, "Scraping", "tabla_grados.csv")
grados=pd.read_csv(ruta_grados)

grados['nombre_grado'] = grados['nombre_grado'].str.split('/')
grados = grados.explode('nombre_grado')
grados['nombre_grado'] = grados['nombre_grado'].str.strip()

ruta_centros = os.path.join(base_dir, "Scraping", "centros_universitarios.csv")
centros=pd.read_csv(ruta_centros)

centros = centros[centros["tipo"].notna()]

def limpiar_universidad(valor):
    if pd.isna(valor):
        return valor
    texto = str(valor)

    if re.search(r'\S+@\S+', texto):
        texto = re.sub(r'^.*?\S+@\S+\s*', '', texto)
    
    texto = re.sub(r'(\(|\)|C/).*$', '', texto)

    return texto.strip()

centros["universidad"] = centros["universidad"].apply(limpiar_universidad)
centros["universidad"] = centros["universidad"].replace(r'\d+', '', regex=True)
centros["universidad"] = centros["universidad"].replace(r'["\']', '', regex=True)


In [330]:
grados.head(2)

,rama,area,nombre_grado,descripcion,salidas,centros
0,Artes y Humanidades,Antropología Social y Cultural,Antropología Social y Cultural,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...,Universidad Nacional de Educación a Distancia ...
1,Artes y Humanidades,Artes,Bellas Artes,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...,Centro de Educación Superior TAI Escuela Unive...


In [331]:
centros.head(2)

,nombre_grado,universidad,tipo,facultad,direccion,telefonos,webs,emails
0,Antropología Social y Cultural,Universidad Nacional de Educación a Distancia ...,Pública,Facultad de Filosofía,"Paseo de la Senda del Rey, 7 (Edificio Humanid...",91 398 69 03,www.uned.es,negociado.alumnos.filosofia@adm.uned.es
1,Bellas Artes,Universidad Antonio de Nebrija,Privada,Facultad de Comunicación y Artes Campus de Mad...,"C/ Santa Cruz de Marcenado, 27 28015 Madrid",91 452 11 03,www.nebrija.com,informa@nebrija.com


# ramas.csv

In [332]:
ramas = pd.DataFrame({
    "id": range(1, len(grados["rama"].unique()) + 1),
    "nombre": grados["rama"].dropna().unique()
})

ramas.to_csv("ramas.csv", index=False, sep=";")

In [333]:
ramas

,id,nombre
0,1,Artes y Humanidades
1,2,Ciencias Sociales y Jurídicas
2,3,Ciencias
3,4,Ciencias de la Salud
4,5,Ingeniería y Arquitectura


# areas.csv

In [334]:
areas = grados[["area", "rama"]].dropna().drop_duplicates().reset_index(drop=True)
areas.insert(0, "id", range(1, len(areas) + 1))
areas = areas.merge(ramas[["id", "nombre"]], left_on="rama", right_on="nombre", how="left")
areas = areas.rename(columns={"nombre": "nombre_rama"})
areas = areas.rename(columns={"id_x": "id", "area": "nombre", "id_y": "id_rama"})
areas = areas[["id", "nombre", "id_rama"]]

areas.to_csv("areas.csv", index=False, sep=";")

In [335]:
areas

,id,nombre,id_rama
0,1,Antropología Social y Cultural,1
1,2,Artes,1
2,3,Diseño,1
3,4,Estudios Culturales,1
4,5,Estudios de Asia y África,1
...,...,...,...
70,71,Ingeniería de la Energía,5
71,72,Ingeniería Matemática,5
72,73,Ingeniería de Materiales,5
73,74,Ingeniería en Telecomunicación,5


# universidad.csv

In [336]:
universidades_unicas = centros["universidad"].dropna().unique()


universidad = pd.DataFrame({
    "id": range(1, len(universidades_unicas) + 1),
    "nombre": universidades_unicas
})

info_universidades = centros.drop_duplicates(subset=["universidad"])[
    ["universidad", "tipo"]
]

universidad = universidad.merge(
    info_universidades,
    left_on="nombre",
    right_on="universidad",
    how="left"
).drop(columns=["universidad"])

universidad.to_csv("universidad.csv", index=False, sep=";")

In [337]:
universidad

,id,nombre,tipo
0,1,Universidad Nacional de Educación a Distancia ...,Pública
1,2,Universidad Antonio de Nebrija,Privada
2,3,Universidad Complutense de Madrid,Pública
3,4,Universidad Francisco de Vitoria,Privada
4,5,Universidad Rey Juan Carlos,Pública
5,6,Universidad Alfonso X El Sabio,Privada
6,7,Universidad San PabloCEU,Privada
7,8,Universidad Europea de Madrid,Privada
8,9,Universidad Carlos III de Madrid,Pública
9,10,Universidad Autónoma de Madrid,Pública


# facultad.csv

In [338]:
facultades_unicas = centros["facultad"].dropna().unique()

facultad = pd.DataFrame({
    "id": range(1, len(facultades_unicas) + 1),
    "nombre": facultades_unicas
})

info_facultades = centros.drop_duplicates(subset=["facultad"])[
    ["facultad", "universidad", "direccion", "telefonos", "webs", "emails"]
]

facultad = facultad.merge(
    info_facultades,
    left_on="nombre",
    right_on="facultad",
    how="left"
).drop(columns=["facultad"])

facultad=facultad.merge(universidad[["id", "nombre"]], left_on="universidad", right_on="nombre")

facultad=facultad[["id_x", "nombre_x", "direccion", "telefonos", "webs", "emails", "id_y"]]
facultad = facultad.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_universidad'})
facultad=facultad.drop_duplicates()

facultad=facultad[["id", "nombre", "id_universidad", "direccion", "telefonos", "webs", "emails"]]

facultad.to_csv("facultad.csv", index=False, sep=";")


In [339]:
facultad

,id,nombre,id_universidad,direccion,telefonos,webs,emails
0,1,Facultad de Filosofía,1,"Paseo de la Senda del Rey, 7 (Edificio Humanid...",91 398 69 03,www.uned.es,negociado.alumnos.filosofia@adm.uned.es
1,2,Facultad de Comunicación y Artes Campus de Mad...,2,"C/ Santa Cruz de Marcenado, 27 28015 Madrid",91 452 11 03,www.nebrija.com,informa@nebrija.com
2,3,Facultad de Bellas Artes Campus de Moncloa,3,"C/ Pintor el Greco, 2 28040 Madrid",91 394 35 60,www.ucm.es,safba@ucm.es
3,4,Facultad de Ciencias de la Comunicación Ctra,4,NaN,91 351 03 03 / 91 709 14 00,www.ufv.es,info@ufv.es
4,5,Facultad de Artes y Humanidades Campus de Aran...,5,"C/ San Pascual, s/n 28300 Aranjuez (Madrid)",91 488 44 21,www.urjc.es,aranjuez.secretariadealumnos@urjc.es
...,...,...,...,...,...,...,...
202,203,Facultad de Informática Campus de Moncloa,3,"C/ Profesor José García Santesmases, 9 Ciudad ...",91 394 75 02,www.ucm.es,salfdi@ucm.es
203,204,Escuela Técnica Superior de Ingeniería de Sist...,17,"C/ Alan Turing, s/n Ctra. de Valencia, km 7 28...",91 067 35 24,www.etsist.upm.es,negociado.alumnos.etsisi@upm.es
204,205,Escuela Técnica Superior de Ingenieros Informá...,17,NaN,91 067 27 24,www.etsist.upm.es,secretaria@fi.upm.es
205,206,Facultad de Informática Campus Moncloa,3,"C/ Profesor José García Santesmases, 9 Ciudad ...",91 394 75 02,www.ucm.es,salfdi@ucm.es


# grados.csv

In [340]:
grados_unicos = grados["nombre_grado"].dropna()

grados_tabla = pd.DataFrame({
    "id": range(1, len(grados_unicos) + 1),
    "nombre": grados_unicos
})

info_grados = grados.drop_duplicates(subset=["nombre_grado"])[
    ["nombre_grado", "area", "descripcion", "salidas"]
]

grados_tabla = grados_tabla.merge(
    info_grados,
    left_on="nombre",
    right_on="nombre_grado",
    how="left"
).drop(columns=["nombre_grado"])

grados_tabla=grados_tabla.merge(areas[["id", "nombre"]], left_on="area", right_on="nombre")
grados_tabla=grados_tabla[["id_x", "nombre_x", "id_y", "descripcion", "salidas"]]
grados_tabla = grados_tabla.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_area'})
grados_tabla=grados_tabla.drop_duplicates()
grados_tabla["id"] = range(1, len(grados_tabla) + 1)



grados_tabla.to_csv("grados.csv", index=False, sep=";")

In [341]:
grados_tabla

,id,nombre,id_area,descripcion,salidas
0,1,Antropología Social y Cultural,1,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
1,2,Antropología Social y Cultural,15,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
2,3,Bellas Artes,2,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...
3,4,Conservación y Restauración del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...
4,5,Artes Escénicas,2,Este grado está adaptado a la realidad profesi...,Estos profesionales tendrán una amplia proyecc...
...,...,...,...,...,...
224,225,Ciencia de Datos e Inteligencia Artificial,75,El grado cubre la creciente necesidad de perfi...,Dichos profesionales estarán capacitados para ...
225,226,Inteligencia Artificial,75,Este Grado tiene como finalidad proporcionar a...,Estratega de entorno digital Especialista en c...
226,227,Computación e Inteligencia Artificial,75,Este Grado tiene como finalidad proporcionar a...,Estratega de entorno digital Especialista en c...
227,228,Ingeniería Robótica Sotware,75,El objetivo de estas titulaciones es formar pr...,La robótica se ha utilizado desde hace años en...


# grados_ofertados.csv

In [342]:
grados_ofertados=centros[["nombre_grado", "facultad"]]

grados_ofertados=grados_ofertados.merge(grados_tabla[["id", "nombre"]], left_on="nombre_grado", right_on="nombre")
grados_ofertados=grados_ofertados[["nombre_grado", "facultad", "id"]]
grados_ofertados=grados_ofertados.merge(facultad[["id", "nombre"]], left_on="facultad", right_on="nombre")
grados_ofertados=grados_ofertados[["id_x", "id_y"]]
grados_ofertados = grados_ofertados.rename(columns={'id_x': 'id_grado', 'id_y': 'id_facultad'})
grados_ofertados=grados_ofertados.drop_duplicates()

grados_ofertados.insert(0, "id", range(1, len(grados_ofertados) + 1))

grados_ofertados.to_csv("grados_ofertados.csv", index=False, sep=";")

In [343]:
grados_ofertados

,id,id_grado,id_facultad
0,1,1,1
1,2,2,1
2,3,52,1
3,4,53,1
4,5,3,2
...,...,...,...
586,531,222,202
588,532,225,204
589,533,228,172
590,534,228,207


In [344]:
centros

,nombre_grado,universidad,tipo,facultad,direccion,telefonos,webs,emails
0,Antropología Social y Cultural,Universidad Nacional de Educación a Distancia ...,Pública,Facultad de Filosofía,"Paseo de la Senda del Rey, 7 (Edificio Humanid...",91 398 69 03,www.uned.es,negociado.alumnos.filosofia@adm.uned.es
1,Bellas Artes,Universidad Antonio de Nebrija,Privada,Facultad de Comunicación y Artes Campus de Mad...,"C/ Santa Cruz de Marcenado, 27 28015 Madrid",91 452 11 03,www.nebrija.com,informa@nebrija.com
2,Bellas Artes,Universidad Complutense de Madrid,Pública,Facultad de Bellas Artes Campus de Moncloa,"C/ Pintor el Greco, 2 28040 Madrid",91 394 35 60,www.ucm.es,safba@ucm.es
3,Bellas Artes,Universidad Francisco de Vitoria,Privada,Facultad de Ciencias de la Comunicación Ctra,NaN,91 351 03 03 / 91 709 14 00,www.ufv.es,info@ufv.es
4,Bellas Artes,Universidad Rey Juan Carlos,Pública,Facultad de Artes y Humanidades Campus de Aran...,"C/ San Pascual, s/n 28300 Aranjuez (Madrid)",91 488 44 21,www.urjc.es,aranjuez.secretariadealumnos@urjc.es
...,...,...,...,...,...,...,...,...
734,Inteligencia Artificial / Computación e Inteli...,Universidad Rey Juan Carlos,Pública,Escuela Técnica Superior de Ingeniería Informá...,"C/ Tulipán, s/n 28933 Móstoles (Madrid)",91 488 70 26 / 91 664 74 35,www.urjc.es,mostoles.secretariadealumnos@urjc.es
737,Inteligencia Artificial / Computación e Inteli...,Universidad Alfonso X El Sabio,Privada,Escuela Politécnica Superior Campus de Villanu...,"Avda. de la Universidad, 1 28691 Villanueva de...",91 810 92 00,www.uax.es,info@uax.es
739,Ingeniería Robótica Sotware,Universidad Carlos III de Madrid,Pública,Escuela Politécnica Superior Campus Leganés,"Avda. de la Universidad, 30 28911 Leganés (Mad...",91 624 60 00,www.uc3m.es,futurosestudiantes@uc3m.es
740,Ingeniería Robótica Sotware,Universidad Rey Juan Carlos,Pública,Escuela Técnica Superior de Ingeniería de Tele...,"Camino del Molino, 5 28942 Fuenlabrada (Madrid)",91 488 72 21,www.urjc.es,etsit.director@urjc.es | etsit.sec.academica@u...
